# YUEHCHAO WU
# Anime Recommender System
# Data Preproccessing

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Source-of-Data-:-Kaggle-Anime-Recommendations-Database" data-toc-modified-id="Source-of-Data-:-Kaggle-Anime-Recommendations-Database-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Source of Data : <a href="https://www.kaggle.com/CooperUnion/anime-recommendations-database" target="_blank">Kaggle Anime Recommendations Database</a></a></span></li><li><span><a href="#Import-libraries" data-toc-modified-id="Import-libraries-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import libraries</a></span></li><li><span><a href="#Load-Anime-&amp;-Anime_rating-csv-into-pandas-Dataframe" data-toc-modified-id="Load-Anime-&amp;-Anime_rating-csv-into-pandas-Dataframe-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load Anime &amp; Anime_rating csv into pandas Dataframe</a></span></li><li><span><a href="#Data-preproccesing" data-toc-modified-id="Data-preproccesing-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Data preproccesing</a></span><ul class="toc-item"><li><span><a href="#Import-My-Custom-Data-Cleaning-Module" data-toc-modified-id="Import-My-Custom-Data-Cleaning-Module-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Import My Custom Data Cleaning Module</a></span></li><li><span><a href="#10-Core-Operation-&amp;-One-Scale-Rater-Elimination-on-Anime_rating-using-clean_data()" data-toc-modified-id="10-Core-Operation-&amp;-One-Scale-Rater-Elimination-on-Anime_rating-using-clean_data()-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>10-Core Operation &amp; One-Scale Rater Elimination on <code>Anime_rating</code> using <code>clean_data()</code></a></span></li><li><span><a href="#Limit-experiment-to-Movie-Comedy-Anime" data-toc-modified-id="Limit-experiment-to-Movie-Comedy-Anime-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Limit experiment to Movie Comedy Anime</a></span></li><li><span><a href="#Get-Final-Form-of-Anime-Rating-Datasets-ready-in-Surprise" data-toc-modified-id="Get-Final-Form-of-Anime-Rating-Datasets-ready-in-Surprise-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Get Final Form of Anime Rating Datasets ready in Surprise</a></span><ul class="toc-item"><li><span><a href="#Data-Splitting-(train-,test)-using-Straitification-on-'user_id'" data-toc-modified-id="Data-Splitting-(train-,test)-using-Straitification-on-'user_id'-4.4.1"><span class="toc-item-num">4.4.1&nbsp;&nbsp;</span>Data Splitting (train ,test) using Straitification on <code>'user_id'</code></a></span></li><li><span><a href="#Load-Final-form-of-Anime-rating--Data-(-final_form_data,-train,-test-)-into-Surprise-Object" data-toc-modified-id="Load-Final-form-of-Anime-rating--Data-(-final_form_data,-train,-test-)-into-Surprise-Object-4.4.2"><span class="toc-item-num">4.4.2&nbsp;&nbsp;</span>Load Final form of Anime rating  Data ( final_form_data, train, test ) into Surprise Object</a></span></li><li><span><a href="#Convert-data-into-surprise-training-sets-&amp;-testing-sets-for-modeling" data-toc-modified-id="Convert-data-into-surprise-training-sets-&amp;-testing-sets-for-modeling-4.4.3"><span class="toc-item-num">4.4.3&nbsp;&nbsp;</span>Convert data into surprise training sets &amp; testing sets for modeling</a></span></li></ul></li></ul></li><li><span><a href="#Summary" data-toc-modified-id="Summary-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Summary</a></span></li></ul></div>

## Source of Data : [Kaggle Anime Recommendations Database](https://www.kaggle.com/CooperUnion/anime-recommendations-database)

## Import libraries

In [64]:
#standard tools
import pandas as pd
from scipy.stats import ttest_rel,probplot
import numpy as np
from matplotlib import pyplot as plt
import re
import glob
import seaborn as sns
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud
import math
import copy as cp
from collections import defaultdict


#from surprise
from surprise import KNNWithMeans,SVD,SVDpp,NMF,SlopeOne,CoClustering
from surprise import Reader, Dataset,accuracy
from surprise.model_selection import cross_validate, KFold, ShuffleSplit, GridSearchCV
from surprise.prediction_algorithms import PredictionImpossible
from surprise.model_selection import train_test_split as surprisesplit
from surprise import dump

#user defined functions
from prec_recall import precision_recall_at_k,pr_eval
from sigweight import KNNSigWeighting
from Rank import *
from data_cleaning import *
from hybrid import WeightedHybrid


## Load Anime & Anime_rating csv into pandas Dataframe

In [14]:
#list of anime files
Alst=glob.glob('anime*') 

#load anime files into list of pandas df
Adf=[]
for i in Alst:
    Adf.append(pd.read_csv(i))

Anime_rating = Adf[1]
Anime = Adf[0]

In [34]:
#remove obesrvations that rating = -1
mask=Anime_rating['rating']!=-1
Anime_rating=Anime_rating[mask]

## Data preproccesing 

### Import My Custom Data Cleaning Module
Codes: [data_cleaning.py](./data_cleaning.py)

In [1]:
from data_cleaning import *

### 10-Core Operation & One-Scale Rater Elimination on `Anime_rating` using `clean_data()` 

In [10]:
#perform eliminaton & core_operation
core=data_cleaner(Anime_rating)

## Iteration 1 ##
There are one-scale raters!
one_Scale_rater_eliminator excuted


There are some user or item has less than 10 ratings!
core_operator excuted!


## Iteration 2 ##
No one scale Rater!


There are some user or item has less than 10 ratings!
core_operator excuted!


## Iteration 3 ##
No one scale Rater!


No item & user has less than 10 ratinngs!


Data is cleaned!


In [293]:
#write 10 core data to csv file 'core_rating.csv'
core.to_csv('core_rating.csv', index=False)

In [11]:
## print summary of  anime rating data after 10 core operation
print('## Summary of Anime Rating Data after cleaning ## \n')
print('Number of observations: {} '.format(len(core)))


for i in core.nunique().index:
    print('Number of Unique {0}s: {1}'.format(i.split('_')[0],core.nunique()[i]))

print('*rating scales: {}'.format(np.sort(core['rating'].unique())))

print('Density: {}'.format(len(core)/(core.nunique()[0]*core.nunique()[1])))

print('='*70)    
print('\n\n## first 10 rows of Anime Rating Data after cleaning ## \n')
print(core.head())
print('='*70)


## Summary of Anime Rating Data after cleaning ## 

Number of observations: 6247010 
Number of Unique users: 54743
Number of Unique animes: 7359
Number of Unique ratings: 10
*rating scales: [ 1  2  3  4  5  6  7  8  9 10]
Density: 0.015506893510584463


## first 10 rows of Anime Rating Data after cleaning ## 

     user_id  anime_id  rating
156        3        20       8
157        3       154       6
158        3       170       9
159        3       199      10
160        3       225       9


### Limit experiment to Movie Comedy Anime

In [82]:
#limit user data to tv & comedy based only
movie_comedy_anime= Anime_subset(Anime,Anime_rating,'Movie','Comedy')


#perform eliminaton & core_operation
movie_comedy_anime_core=data_cleaner(movie_comedy_anime)

## Iteration 1 ##
There are one-scale raters!
one_Scale_rater_eliminator excuted


There are some user or item (if not None) has less than 10 ratings!
core_operator excuted!


## Iteration 2 ##
There are one-scale raters!
one_Scale_rater_eliminator excuted


There are some user or item (if not None) has less than 10 ratings!
core_operator excuted!


## Iteration 3 ##
No one scale Rater!


There are some user or item (if not None) has less than 10 ratings!
core_operator excuted!


## Iteration 4 ##
No one scale Rater!


No item & user has less than 10 ratinngs!


Data is cleaned!


In [83]:
## size &info
movie_comedy_anime_core.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 184949 entries, 162 to 7813429
Data columns (total 3 columns):
user_id     184949 non-null int64
anime_id    184949 non-null int64
rating      184949 non-null int64
dtypes: int64(3)
memory usage: 5.6 MB


In [ ]:
## print summary of  Movie & comedy based Anime Rating Data after cleaning ## \n')
print('Number of observations: {} '.format(len(movie_comedy_anime_core)))


for i in movie_comedy_anime_core.nunique().index:
    print('Number of Unique {0}s: {1}'.format(i.split('_')[0],movie_comedy_anime_core.nunique()[i]))

print('*rating scales: {}'.format(np.sort(movie_comedy_anime_core['rating'].unique())))

print('Density: {}'.format(len(movie_comedy_anime_core)/(movie_comedy_anime_core.nunique()[0]*movie_comedy_anime_core.nunique()[1])))

print('='*80)    
print('\n\n## first 10 rows of  TV & comedy based Anime Rating Data after cleaning ## \n')
print(movie_comedy_anime_core.head())
print('='*80)

In [85]:
#write to csv
movie_comedy_anime_core.to_csv('core_movie_comedy_rating.csv', index=False)

### Get Final Form of Anime Rating Datasets ready in Surprise

#### Data Splitting (train ,test) using Straitification on `'user_id'`
using straitification on user to make sure each user apear in both train and test set as stated proportion



In [22]:
#load final form of anime rating data 'core_movie_comedy_rating.csv'
final_form_data=pd.read_csv('core_movie_comedy_rating.csv')

#splittig on final form of data
train, test = train_test_split(final_form_data, test_size=0.2, random_state=1, 
                               stratify=final_form_data['user_id'])

##write train & test to csv
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

#### Load Final form of Anime rating  Data ( final_form_data, train, test ) into Surprise Object

In [2]:
## load data into suprise object ##
reader = Reader(line_format='user item rating', sep=',',rating_scale=(1, 10),skip_lines=1)
WholeSet = Dataset.load_from_file('core_movie_comedy_rating.csv', reader)
S_train = Dataset.load_from_file('train.csv', reader)
S_test = Dataset.load_from_file('test.csv', reader)

####  Convert data into surprise training sets & testing sets for modeling

In [3]:
# surprise trainset  
trainset = S_train.build_full_trainset() # 80% of data for training
WholeSet_train = WholeSet.build_full_trainset() # wholeSet as training


In [4]:
# testset for evaluating RMSE

testset = list(map(lambda x : x[0:3],S_test.raw_ratings)) # 20% of data as testset for evaluating rmse


In [5]:
# testset for evaluating relevence(Precision, Recall, NDCG)
NoRatingSet = WholeSet_train.build_anti_testset(fill=1) #whole unknown rating

_, noRate_test = train_test_split(NoRatingSet, test_size=0.2, random_state=1, stratify=list(map(lambda x: x[0], NoRatingSet)))

testPlusUnknown=testset+noRate_test #testset + 20% of unknown rating for evaluating relevence



## Summary

In [63]:
#read report 
data_ccompare=pd.read_csv('data_ccompare.csv',index_col=0)
data_ccompare.style.highlight_min(data_ccompare.columns[:-1]).highlight_max(['Density'])

,Number of Rating,Number of user,Number of Item,Missing value (Rating = -1),Number of Item less than 10 rating,Number of User less than 10 rating,Number of user that only rate with one value,Density
Original Anime Rating data,7813737,73515,11200,1476496,2562,14482,5476,0.0092
Final Form of Anime Rating data,184949,8905,334,0,0,0,0,0.0622


__*Comment:__

After filtering out all the perfect & the worst rating user/item, and applying 10 core operation on the anime rating table,  the __density__ of  final form of anime rating data increase from __~0.009 to ~ 0.016__
    

In [59]:
#save reprot as csv
data_ccompare.to_csv('data_ccompare.csv')


In [12]:
#summary of data that will be used in surprise

print('## Final form of Anime (Comedy & movie based) rating data ##\n')
print('Number of Observation: {}'.format(len(WholeSet.raw_ratings)))
print('Number of Users: {}'.format(WholeSet_train.n_users))
print('Number of items: {}'.format(WholeSet_train.n_items))
print('Density: {}'.format(WholeSet_train.n_ratings/(WholeSet_train.n_users*WholeSet_train.n_items)))
print('='*110)
print('\n')



print('## Surprise trainset  object (80% of whole data) ##\n')
print('Object Type: {}'.format(type(trainset)))
print('Number of observations: {}'.format(len(S_train.raw_ratings)))
print('Number of Users ( trainset.n_users ): {}'.format(trainset.n_users))
print('Number of items ( trainset.n_items ): {}'.format(trainset.n_items))
print('='*110)
print('\n')

print('## Holdeout Testset (20% of whole data) for evalauting Rating Prediction performance##\n')

print('Object Type: {}'.format(type(testset)))
print('Number of Observation: {}'.format(len(S_test.raw_ratings)))
print('Number of Users: {}'.format(len(np.unique(np.array(testset)[:,0],return_counts=True)[1])))
print('Number of items: {}'.format(len(np.unique(np.array(testset)[:,1],return_counts=True)[1])))
print('='*110)
print('\n')


print('## Testset (Holdeout testset + 20% unknown rating data) for evalauting Reccomendation performance ##\n')
print('Object Type: {}'.format(type(testPlusUnknown)))
print('Number of Observation: {}'.format(len(testPlusUnknown)))
print('Number of Users: {}'.format(len(np.unique(np.array(testPlusUnknown)[:,0],return_counts=True)[1])))
print('Number of items: {}'.format(len(np.unique(np.array(testPlusUnknown)[:,1],return_counts=True)[1])))
print('='*110)
print('\n')



## Final form of Anime (Comedy & movie based) rating data ##

Number of Observation: 184949
Number of Users: 8905
Number of items: 334
Density: 0.06218298943942547


## Surprise trainset  object (80% of whole data) ##

Object Type: <class 'surprise.trainset.Trainset'>
Number of observations: 147959
Number of Users ( trainset.n_users ): 8905
Number of items ( trainset.n_items ): 334


## Holdeout Testset (20% of whole data) for evalauting Rating Prediction performance##

Object Type: <class 'list'>
Number of Observation: 36990
Number of Users: 8905
Number of items: 329


## Testset (Holdeout testset + 20% unknown rating data) for evalauting Reccomendation performance ##

Object Type: <class 'list'>
Number of Observation: 594855
Number of Users: 8905
Number of items: 334


